In [ ]:
pip install -U sentence-transformers

In [2]:
# Packages

#pip install -U sentence-transformers
from termcolor import colored
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import numpy as np
import string
import regex as re
import zipfile
import gc
from scipy.stats import boxcox
import sys 
from collections import Counter 
from tqdm import tqdm 
import pandas as pd 
import numpy as np 
import warnings, math
from termcolor import colored
import pickle
import string
from sklearn.metrics.pairwise import cosine_similarity

# for eval
from sklearn.model_selection import train_test_split
import random


# for SVD
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix
pd.set_option('display.float_format','{:.5f}'.format)

# for EMB
from sentence_transformers import SentenceTransformer, util
import scipy
from sklearn import preprocessing 

# for TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy
from sklearn import preprocessing 

# for creating a network
from sklearn.cluster import KMeans 

In [4]:
EXT_FEATURES_PATH = 'D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\Graph_based\ext_features_agg.csv'
NEW_DF_PATH = 'D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\Graph_based\Oct_28_df.csv'

ext_features = pd.read_csv(EXT_FEATURES_PATH)
df = pd.read_csv(NEW_DF_PATH)


### Create Embeddings

In [ ]:
projects_df = projects_df[projects_df['Project ID'].isin(df['Project ID'])].reset_index(drop=True)

project_txt = projects_df.loc[:, 'project_txt']
projects_id = projects_df['Project ID'].tolist()


In [ ]:
# ----------------- embeddings---------------------#
model = SentenceTransformer('paraphrase-distilroberta-base-v1') # , device= 'cuda'
embeddings = model.encode(project_txt, batch_size=256, show_progress_bar=True)

In [ ]:
with open(EMBEDDING_PATH, 'wb') as f:
    pickle.dump(embeddings, f)

### Donor Profile

In [5]:
EMBEDDING_PATH = 'D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\Graph_based\Oct_28_project_embeddings.pickle'
project_embeddings = pd.read_pickle(EMBEDDING_PATH)

In [7]:
# ==================== Embeddings ==================== #

# ----------------------- projects profiles------------------- #
def get_project_profile_emb(project_id: str, project_embeddings):

    # get the ids
    idx = projects_id.index(project_id)
    project_profile = project_embeddings[idx:idx+1]
    
    return project_profile



def get_projects_profiles_emb(ids: pd.Series, project_embeddings):


    profiles_list = [get_project_profile_emb(project_id, project_embeddings)[0] for project_id in np.ravel([ids])]
    project_profiles = np.vstack(profiles_list)

    return project_profiles


# ----------------------- Donors profiles------------------- #

def build_donors_profile_emb(donor_id: str, df_train_donor_index: pd.DataFrame):

    # get the id of each person and the projects they
    # donated to
    donations_donor_df = df_train_donor_index.loc[donor_id]


    # get the vectors of projects this person has donated to
    donor_donated_project_intrain_profiles = get_projects_profiles_emb(donations_donor_df['Project ID'], project_embeddings)


    # get the smoothed donated amount as the weight of each project
    donor_project_strengths = np.array(donations_donor_df['Donation Amount']).reshape(-1, 1)


    # multiply the weights and tfidf vectors
    multiplication = np.multiply(donor_donated_project_intrain_profiles, donor_project_strengths)


    # now we normalize the whole vector 
    normalized_donor_preference = preprocessing.normalize(np.sum(multiplication, axis=0).reshape(1, -1))
    


    return normalized_donor_preference


def build_donors_profiles_emb(df_test_donor_index: pd.DataFrame, df_train_donor_index: pd.DataFrame, sample_size: int = None):
    
    # now for all donors we build a profile in a dictionary
    donor_profiles = {}
    donors_in_test_set = df_test_donor_index.index.unique().tolist()[:sample_size]

    for donor_id in tqdm(donors_in_test_set[:sample_size], position=0, leave=True):
        donor_profiles[donor_id] = build_donors_profile_emb(donor_id, df_train_donor_index)

    return donor_profiles

    



In [ ]:
donor_profiles_emb = build_donors_profiles_emb(df_test_donor_index, df_train_donor_index)